# Neural Style Transfer (2)
Xun Huang et. al Arbitrary Style Transfer in Real-time with Adaptive Instance Normalization, ICCV 2017


## Import Libraries
필요한 라이브러리들을 가져옵니다.

In [1]:
%pylab inline


import numpy as np
import gc
import visdom
import os
import time
import numpy as np
from os import listdir
from PIL import Image
from datetime import datetime
import ipdb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torchvision import utils, transforms, models
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

Populating the interactive namespace from numpy and matplotlib


## Mean & Std
PyTorch에서 평균과 표준편차는 다음 방법으로 쉽게 계산할 수 있습니다.

`torch.randn`은 $X\sim\mathcal{N}(0,1)$에서 샘플링하는 함수입니다.(Normal distribution)

`torch.rand`은 $Y\sim\mathcal{U}[0,1)$에서 샘플링하는 함수입니다.(Uniform sampling)

In [2]:
x = torch.randn(10)
print("x: {}".format(x))
print("Mean of x: {}".format(torch.mean(x)))
print("Std of x: {}".format(torch.std(x)))

y = torch.rand(10)
print("y: {}".format(y))
print("Mean of y: {}".format(torch.mean(y)))
print("Std of y: {}".format(torch.std(y)))

x: tensor([-1.4047,  0.3893, -0.0193, -0.3713,  0.8378,  0.1992, -0.8536, -1.1079,
        -1.2084, -0.4172])
Mean of x: -0.3956086337566376
Std of x: 0.747566819190979
y: tensor([0.6744, 0.5997, 0.1499, 0.0189, 0.4490, 0.9363, 0.4462, 0.2701, 0.0557,
        0.5478])
Mean of y: 0.4147930145263672
Std of y: 0.29252907633781433


만약 $x$가 Tensor이고 사용자가 원하는 dimension에 대해서만 평균 및 표준편차를 계산하고 싶을때

In [3]:
x = torch.randn(2,3,224,224)
print("Mean of x for all dims: {}".format(torch.mean(x)))
print("Std of x for all dims: {}".format(torch.std(x)))
print("Mean of x for dim [2,3]: {}".format(torch.mean(x, [2,3])))
print("Std of x for dim [2,3]: {}".format(torch.std(x, [2,3])))

Mean of x for all dims: 0.00037972815334796906
Std of x for all dims: 1.000279188156128
Mean of x for dim [2,3]: tensor([[ 0.0009, -0.0017,  0.0044],
        [ 0.0012, -0.0009, -0.0017]])
Std of x for dim [2,3]: tensor([[0.9967, 1.0048, 1.0001],
        [0.9984, 0.9988, 1.0030]])


## Normalization
$X$의 평균이 $\mu$이고 표준편차가 $\sigma$일때 다음과 같이 정규화 가능합니다.
$$z = \frac{x-\mu}{\sigma}$$

위에서 사용한 `torch.mean`과 `torch.std`를 이용하여 다음과 같이 $z$를 계산하고 평균과 표준편차를 계산해보겠습니다.

In [4]:
x = torch.rand(2,3,224,224)
print("Mean of x: {}".format(torch.mean(x, [2,3])))
print("Std of x: {}".format(torch.std(x, [2,3])))

z=(x - torch.mean(x, [2,3]))/torch.std(x, [2,3])
print("Mean of z: {}".format(torch.mean(z, [2,3])))
print("Std of z: {}".format(torch.std(z, [2,3])))

Mean of x: tensor([[0.5000, 0.4994, 0.5003],
        [0.4985, 0.5014, 0.4999]])
Std of x: tensor([[0.2886, 0.2888, 0.2883],
        [0.2892, 0.2898, 0.2890]])


RuntimeError: The size of tensor a (224) must match the size of tensor b (3) at non-singleton dimension 3

multiplication과 division에서 dimension이 맞지 않을 경우 위와 같은 에러를 확인할 수 있습니다.

평균과 표준편차를 계산하면서 차원이 감소해서 발생한 문제입니다.

`keepdim`이라는 인자를 사용하여 차원 감소 없이 평균과 표준편차를 계산할 수 있습니다.

In [5]:
x = torch.rand(2,3,224,224)
print("Mean of x: {}".format(torch.mean(x, [2,3])))
print("Std of x: {}".format(torch.std(x, [2,3])))

z=(x - torch.mean(x, [2,3], keepdim=True))/torch.std(x, [2,3], keepdim=True)
print("Mean of z: {}".format(torch.mean(z, [2,3])))
print("Std of z: {}".format(torch.std(z, [2,3])))

Mean of x: tensor([[0.5014, 0.5014, 0.5014],
        [0.5009, 0.4994, 0.5010]])
Std of x: tensor([[0.2886, 0.2882, 0.2881],
        [0.2888, 0.2882, 0.2881]])
Mean of z: tensor([[ 3.5307e-07,  3.4136e-08,  1.3609e-07],
        [-6.4812e-08, -1.7114e-07,  1.0035e-08]])
Std of z: tensor([[1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000]])


In [8]:
x = torch.rand(2,3,224,224)
print(x.shape)
print(torch.mean(x).shape)
print(torch.mean(x, [2,3]).shape)

torch.Size([2, 3, 224, 224])
torch.Size([])
torch.Size([2, 3])
